In [ ]:
import csv
import os
import pandas as pd
import re
import time
from datetime import datetime
from pathlib import *
import random
from bs4 import BeautifulSoup
from unidecode import unidecode
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pytz
from pyvirtualdisplay import Display
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import socket

In [ ]:
# Start a virtual display
display = Display(visible=0, size=(1920, 1080))
display.start()


def time_now():
  # Define the Vietnam timezone
  local_tz = pytz.timezone('Asia/Ho_Chi_Minh')
  # Get the current time in UTC
  now_utc = datetime.now(pytz.utc)
  # Convert UTC time to local time
  now = now_utc.astimezone(local_tz).date().strftime('%Y-%m-%d')
  return now, now_utc, local_tz


# Chrome
def chrome_drive():

    # Configure Chrome options
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration
    options.add_argument('--no-sandbox')  # Bypass OS security model
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

    # Create a driver instance
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


    # Disable the "Chrome is being controlled by automated test software" notification
    options.add_experimental_option("excludeSwitches", ["enable-automation"])

    # # Disable the "navigator.webdriver" property
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

    # Disable the "Chrome is being controlled by automated test software" banner
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Maximize the window to avoid fingerprinting based on screen resolution
    options.add_argument("start-maximized")

    # Disabling the Automation Extension can help prevent detection as an automated script and increase the chances of
    # successfully completing your automation tasks.
    options.add_experimental_option('useAutomationExtension', False)

    # This argument tells the browser to ignore any SSL certificate errors that may occur while accessing a website,
    # which is useful when testing on a site with a self-signed or invalid SSL certificate. Without this argument,
    # the browser will display a warning message about the certificate and require manual confirmation to proceed.
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors=yes')

    # wait for the page to be fully loaded before proceeding
    # options.page_load_strategy = 'normal'  # 'none', 'eager', or 'normal'

    prefs={
        "disable-transitions": True,
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2
    }
    options.add_experimental_option("prefs", prefs)

    return driver

In [ ]:
class CellPhoneS:

    def ensure_internet_connection(self):
        
        def is_internet_connected():
            try:
                socket.create_connection(("8.8.8.8", 53), timeout = 5)
                return True
            except OSError:
                return False


        max_attempts=300
        retry_interval=5

        # Attempt to establish internet connection
        for attempt in range(1, max_attempts + 1):
            if is_internet_connected():
                return True
            else:
                print(
                    f"No internet connection. Retrying in {retry_interval} seconds... (Attempt {attempt}/{max_attempts})")
                time.sleep(retry_interval)

        # If internet connection couldn't be established, wait for disruption and a specific duration
        print(f"No internet connection after {max_attempts} attempts. Waiting for disruption...")
        while is_internet_connected():
            time.sleep(1)

        return True

    def cps(self, link_check: list, restart_link: str):

        driver=chrome_drive()
        wait=WebDriverWait(driver, 10)

        # Define the base path to Google Drive folder
        now, now_utc, local_tz = time_now()
        base_path='../content'
        if not os.path.exists(base_path):
            os.makedirs(base_path)
        output_dir=os.path.join(base_path, f"{now}")
        output_img=os.path.join(output_dir, 'img_cps')

        def record():
            output_dir=os.path.join(base_path, f"{now}")

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            file_path=os.path.join(output_dir, f"6-cps{now}.csv")

            with open(file_path, "a") as file:
                writer=csv.DictWriter(file,
                                      fieldnames = ["Product_Name", "Ton_Kho", "Gia_Niem_Yet", "Gia_Khuyen_Mai", "Date",
                                                    "Khuyen_Mai","Thanh_Toan", "Link", "screenshot_name"], delimiter = ";")
                if os.stat(file_path).st_size == 0:
                    writer.writeheader()
                for row in data_list:
                    writer.writerow(row)

        def screen_shot(product_name):
            now, now_utc, local_tz = time_now()
            if not os.path.exists(output_img):
                os.makedirs(output_img)
            product_name_new=re.sub(r'[^\w\-\.]', '_', product_name).strip('. ')
            driver.fullscreen_window()
            driver.set_window_size(1920, 2080)
            # Scroll up by 100 pixels
            # driver.execute_script("window.scrollBy(0, -100);")
            driver.get_screenshot_as_png()
            product_name_new=f"{output_img}/{product_name_new}_{now_utc.astimezone(local_tz)}.png"
            driver.save_screenshot(product_name_new)
            return product_name_new.replace(f"{output_img}/", "")

        def check_price():
            try:
                popup=wait.until(
                    EC.visibility_of_element_located((By.XPATH, "//div[@id='popup-modal']/a[2]")))
                driver.execute_script("arguments[0].scrollIntoView(true);", popup)
                driver.execute_script("arguments[0].click();", popup)
                time.sleep(2)
            except TimeoutException:
                    pass

            try:
                driver.find_element(By.CLASS_NAME, "box-name__box-product-name")
                # driver.find_element(By.TAG_NAME, "h1")
            except NoSuchElementException:
                try:
                    product_name = driver.find_element(By.CLASS_NAME, "box-product-name").text
                except NoSuchElementException:
                    product_name = driver.find_element(By.TAG_NAME, "h1").text
            else:
                product_name = driver.find_element(By.CLASS_NAME, "box-name__box-product-name").text
                
            to_remove_in_name = ["Chính hãng", " I ", " | ", " VN/A", " Apple Việt Nam", "Chính Hãng"]
            for item in to_remove_in_name:
                if item in product_name:
                    product_name = product_name.replace(item, "")
            print(product_name)

            # 2 Check the inventory
            try:
                driver.find_element(By.CSS_SELECTOR, ".order-button strong").text.lower()
            except NoSuchElementException:
                ton_kho = "No"
            else:
                ton_kho_raw = driver.find_element(By.CSS_SELECTOR, ".order-button strong").text.lower()
                if "mua ngay" in ton_kho_raw:
                    ton_kho = "Yes"
                elif "sắp về hàng" in ton_kho_raw:
                    ton_kho = "No"
                elif "đặt trước" in ton_kho_raw:
                    ton_kho = "Pre-order"
                else:
                    ton_kho = "Please check again"

            # 3 Check the gia_khuyen_mai
            try:
                driver.find_element(By.XPATH, "//p[@class='product__price--show']")
            except NoSuchElementException:
                try:
                    driver.find_element(By.CSS_SELECTOR, ".list-linked .active span")
                except NoSuchElementException:
                    try:
                        driver.find_element(By.XPATH, "(//div[contains(@class, 'active')])/p[@class='tpt---sale-price']")
                    except NoSuchElementException:
                        gia_khuyen_mai = 0
                    else:
                        gia_khuyen_mai = driver.find_element(By.XPATH, "(//div[contains(@class, 'active')])/p[@class='tpt---sale-price']").text
                else:
                    gia_khuyen_mai = driver.find_element(By.CSS_SELECTOR, ".list-linked .active span").text
            else:
                # gia_khuyen_mai = driver.find_element(By.CSS_SELECTOR, ".box-info__box-price .product__price--show").text
                gia_khuyen_mai = driver.find_element(By.XPATH, "//p[@class='product__price--show']").text
            print(gia_khuyen_mai)

            # 4 Check the gia_niem_yet
            try:
                driver.find_element(By.XPATH,
                "//div[@class='box-detail-product__box-center column']/div/div/p[@class='product__price--through']")
            except NoSuchElementException:
                gia_niem_yet = gia_khuyen_mai
            else:
                gia_niem_yet = driver.find_element(By.XPATH,
                "//div[@class='box-detail-product__box-center column']/div/div/p[@class='product__price--through']").text

            # 5 Check khuyen_mai
            try:
                driver.find_element(By.CLASS_NAME, "box-product-promotion-content")
            except NoSuchElementException:
                khuyen_mai = ""
            else:
                khuyen_mai = driver.find_element(By.CLASS_NAME, "box-product-promotion-content")\
                    .text.replace('\n\n', "")

            # 6 Check thanh_toan
            try:
                driver.find_element(By.CLASS_NAME, "box-more-promotion")
            except NoSuchElementException:
                thanh_toan = ""
            else:
                thanh_toan = driver.find_element(By.CLASS_NAME, "box-more-promotion").text.replace('\n\n', "")

            screenshot_name = screen_shot(product_name)
            new_data = {"Product_Name": product_name, "Ton_Kho": ton_kho, "Gia_Niem_Yet": gia_niem_yet,
                        "Gia_Khuyen_Mai": gia_khuyen_mai, "Date": now, "Khuyen_Mai": khuyen_mai,
                        "Thanh_Toan": thanh_toan, 'Link': link, 'screenshot_name': screenshot_name}
            data_list.append(new_data)

        self.link_check=link_check
        self.restart_link=restart_link

        for link in link_check[link_check.index(restart_link):]:
            print(link)
            data_list=[]
            try:
                self.ensure_internet_connection()
                driver.get(link)
                time.sleep(10)
                check_price()
                record()
            except TimeoutException:
                print(datetime.now())
                print(f"Start again from link: {link}")
                self.restart_link = link
                self.cps(self.link_check, self.restart_link)
            except NoSuchElementException:
                if driver.find_element(By.XPATH, "//div[@class='not-found']"):
                    continue
        driver.quit()
